In [1]:
from google.colab import files
uploaded = files.upload()

Saving covid_caso.csv to covid_caso.csv


In [5]:
import pandas as pd
import io
df2 = pd.read_csv(io.BytesIO(uploaded['covid_caso.csv']))
# Dataset is now stored in a Pandas Dataframe

In [6]:
df2.head()

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-11-03,AP,NaN,state,52832,751,228,True,845731.0,861773.0,16.0,6130.61676,0.0142
1,2020-11-02,AP,NaN,state,52653,749,227,False,845731.0,861773.0,16.0,6109.84563,0.0142
2,2020-11-01,AP,NaN,state,52500,748,226,False,845731.0,861773.0,16.0,6092.09154,0.0142
3,2020-10-31,AP,NaN,state,52175,748,225,False,845731.0,861773.0,16.0,6054.37859,0.0143
4,2020-10-30,AP,NaN,state,52030,747,224,False,845731.0,861773.0,16.0,6037.55281,0.0144


In [19]:
df2.dropna(subset = ["state"], inplace=True)
df2.dropna(subset = ["city"], inplace=True)
df2.dropna(subset = ["confirmed"], inplace=True)
df2.dropna(subset = ["deaths"], inplace=True)

In [26]:
df2_ap = df2[df2['state'] == 'AP']
df2_ap.city.value_counts()

Macapá                     224
Oiapoque                   209
Santana                    208
Porto Grande               199
Mazagão                    199
Vitória do Jari            199
Laranjal do Jari           198
Serra do Navio             191
Pedra Branca do Amapari    190
Itaubal                    189
Amapá                      187
Tartarugalzinho            187
Calçoene                   182
Ferreira Gomes             181
Cutias                     178
Pracuúba                   178
Name: city, dtype: int64

In [80]:
states = df2.state.unique()
states

array(['AP', 'AC', 'AM', 'DF', 'AL', 'CE', 'ES', 'BA', 'MS', 'MA', 'GO',
       'MT', 'PA', 'PB', 'PE', 'MG', 'PI', 'RO', 'RJ', 'RN', 'PR', 'RR',
       'SE', 'SC', 'RS', 'TO', 'SP'], dtype=object)

In [100]:
state_cities_dict = {}
# for state in states:
df2.loc[ (df2['state'] == "AL") & (df2['place_type'] == 'city')]

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
20792,2020-11-07,AL,Anadia,city,504,8,199,True,17545.0,17526.0,2700201.0,2875.72749,0.0159
20793,2020-11-06,AL,Anadia,city,503,8,198,False,17545.0,17526.0,2700201.0,2870.02168,0.0159
20794,2020-11-05,AL,Anadia,city,503,8,197,False,17545.0,17526.0,2700201.0,2870.02168,0.0159
20795,2020-11-04,AL,Anadia,city,503,8,196,False,17545.0,17526.0,2700201.0,2870.02168,0.0159
20796,2020-11-03,AL,Anadia,city,503,8,195,False,17545.0,17526.0,2700201.0,2870.02168,0.0159
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40139,2020-06-07,AL,Água Branca,city,4,0,5,False,20196.0,20230.0,2700102.0,19.77261,0.0000
40140,2020-06-06,AL,Água Branca,city,4,0,4,False,20196.0,20230.0,2700102.0,19.77261,0.0000
40141,2020-06-05,AL,Água Branca,city,1,0,3,False,20196.0,20230.0,2700102.0,4.94315,0.0000
40142,2020-06-04,AL,Água Branca,city,1,0,2,False,20196.0,20230.0,2700102.0,4.94315,0.0000


In [129]:
column_names = ['state', 'city', 'confirmed', 'deaths', 'estimated_population', 'death_rate', 'city_ibge_code']
df_processed = pd.DataFrame(columns = column_names)

city_ibge_code_list = df2.city_ibge_code.unique()
for city_ibge_code in city_ibge_code_list:
  tmp = df2.loc[ (df2['city_ibge_code'] == city_ibge_code) & (df2['place_type'] == 'city') ]
  if( len(tmp['state'].unique()) == 1):
    df_processed = df_processed.append({
        'state': tmp['state'].unique()[0],
        'city': tmp['city'].unique()[0],
        'confirmed': tmp['confirmed'].max(),
        'deaths': tmp['deaths'].max(),
        'estimated_population': tmp['estimated_population'].max(),
        'death_rate': tmp['death_rate'].max(),
        'city_ibge_code': tmp['city_ibge_code'].unique()[0]
        }, ignore_index=True)

In [130]:
# df_processed['state'] = tmp['state'].unique()[0]
#     df_processed['city'] = tmp['city'].unique()[0]
#     df_processed['confirmed'] = tmp['confirmed'].max()
#     df_processed['deaths'] = tmp['deaths'].max()
#     df_processed['estimated_population'] = tmp['estimated_population'].max()
#     df_processed['death_rate'] = tmp['death_rate'].max()
#     df_processed['city_ibge_code'] = tmp['city_ibge_code'].unique()[0]

# tmp = df2.loc[ (df2['city_ibge_code'] == 1200013.0) & (df2['place_type'] == 'city') ]
# tmp['state'].unique()[0]
# tmp['city'].unique()[0]
# tmp['confirmed'].max()
# tmp['deaths'].max()
# tmp['estimated_population'].max()
# tmp['death_rate'].max()
# tmp['city_ibge_code'].unique()[0]
df_processed.head()

,state,city,confirmed,deaths,estimated_population,death_rate,city_ibge_code
0,AP,Amapá,787,4,9187.0,0.1000,1600105.0
1,AP,Calçoene,1202,5,11306.0,0.1250,1600204.0
2,AP,Cutias,620,2,6101.0,0.0333,1600212.0
3,AP,Ferreira Gomes,590,3,7967.0,0.3333,1600238.0
4,AP,Itaubal,321,0,5617.0,0.0000,1600253.0


In [131]:
from google.colab import files
df_processed.to_csv('covid_df_processed.csv') 
files.download('covid_df_processed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [142]:
states_column_names = ['state', 'confirmed', 'deaths', 'estimated_population']
df_states_processed = pd.DataFrame(columns = states_column_names)

In [143]:
states = df2.state.unique()
for state in states:
  tmp = df_processed.loc[df_processed['state'] == state]
  df_states_processed = df_states_processed.append({
        'state': tmp['state'].unique()[0],
        'confirmed': tmp['confirmed'].sum(),
        'deaths': tmp['deaths'].sum(),
        'estimated_population': tmp['estimated_population'].sum()
        }, ignore_index=True)

In [145]:
df_states_processed['death_rate'] = df_states_processed['deaths'] / df_states_processed['confirmed']

In [146]:
df_states_processed

,state,confirmed,deaths,estimated_population,death_rate
0,AP,52653,749,861773.0,0.0142252
1,AC,31327,698,894470.0,0.0222811
2,AM,162139,4571,4207714.0,0.0281919
3,DF,189277,3439,3055149.0,0.0181691
4,AL,91617,2266,3351543.0,0.0247334
5,CE,273857,9414,9187103.0,0.0343756
6,ES,127558,3472,4064052.0,0.027219
7,BA,351877,7678,14930634.0,0.0218201
8,MS,82094,1588,2809394.0,0.0193437
9,MA,187302,4114,7114598.0,0.0219645


In [148]:
from google.colab import files
df_states_processed.to_csv('covid_states_df_processed.csv')
files.download('covid_states_df_processed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>